# AllegroGraph ETL

If you are connecting in a regular jupyter notebook and have activated our `ag-tutorial` environment then you do not need to run the following cell to install `agraph-python` and `pandas`. However, if you are in a **Google Colab** please run the following cell.

In [ ]:
%pip install agraph-python pycurl

## Setup
We start by setting the environment variables based on your server installation.
If you are connecting on a cloud server, you do not need to change `AGRAPH_PORT` or `AGRAPH_USER`, but you do need to change `AGRAPH_HOST` and `AGRAPH_PASSWORD` to fit your server. We have left some variables in place as an example of what to look for, but they need to be replaced with your values.
If you are connecting to a local installation the host will most likely be `localhost` and whatever port you installed AllegroGraph on (standard is `10035`). Then your `AGRAPH_USER` and `AGRAPH_PASSWORD` need to be added.

In [10]:
import os

os.environ['AGRAPH_HOST'] = 'https://ag197y8xsj2epl2e.allegrograph.cloud' #add your AllegroGraph Cloud url here (starting with https, copy till the end of allegrograph.cloud)
os.environ['AGRAPH_PORT'] = '443' #Agraph is listening at port 443, you do not need to change this if you are connected to a cloud server
os.environ['AGRAPH_USER'] = 'admin' #your username should be 'admin', you do not need to change this if you are connected to a cloud server
os.environ['AGRAPH_PASSWORD'] = 'GrMEKDvQFaN2bkrHJeiCbv' #Add your password here as a string

Then we import the required packages

In [1]:
from franz.openrdf.connect import ag_connect
from franz.openrdf.query.query import QueryLanguage
from franz.openrdf.vocabulary import RDF, RDFS
import pandas as pd
from pprint import pprint

Create a new respository (or we clear it if it already exists) and set the namespaces both in the local python environment and on the server

In [12]:
conn = ag_connect('agraph-etl', clear=True)

f = conn.namespace('http://franz.com/example-etl/') #sets the namespace in the local python environment
conn.setNamespace('f', 'http://franz.com/example-etl') #sets the namespace on the server

A quick utility class to most efficiently add large amounts of triples.

In [13]:
class BufferTriples:
    def __init__(self, conn, max_size=10000):
        self.conn = conn
        self.buffer_triples = []
        self.max_size = max_size
        
    def add_triple_to_buffer(self, triple):
        if len(self.buffer_triples) < self.max_size:
            self.buffer_triples.append(triple)
        else:
            self.conn.addTriples(self.buffer_triples)
            self.buffer_triples = [triple]
            
    def flush_triples(self):
        self.conn.addTriples(self.buffer_triples)
        self.buffer_triples=[]

# ETL on CSV/XLSX Files and/or SQL Tables

Here we will import some CSV files and read them into Pandas dataframes and then convert them to triples. Even though these dataframes will be very small, this same method would work for much larger datasets as well.

In [4]:
#beings_df = pd.read_csv('data/beings.csv')
beings_df = pd.read_csv('https://raw.githubusercontent.com/franzinc/agraph-tutorials/master/data/beings.csv')

In [5]:
beings_df.head()

,id,name,age,race
0,1,Aragorn,87,Dúnedain
1,2,Frodo,50,Hobbit
2,3,Legolas,2931,Elf
3,4,Gimli,140,Dwarf
4,5,Gandalf,2000,Wizard


First we create a buffer that will allow us to add triples to the server whenever we so desire. 

Then we loop through the dataframe and create a unique URI based on the being's `id` column. Then we attach all other metadata to that newly created URI by adding triples to the buffer. At the end of this process we "flush" the triples, meaning we add the triples in the buffer to the server. (Meaning the buffer is now empty, but still exists in memory)

In [16]:
buffer = BufferTriples(conn)
for index, row in beings_df.iterrows():
    being_uri = conn.createURI(f"http://franz.com/example-being/{str(row['id'])}")
    buffer.add_triple_to_buffer((being_uri, RDF.TYPE, f.Being))
    buffer.add_triple_to_buffer((being_uri, f.name, row['name']))
    buffer.add_triple_to_buffer((being_uri, RDFS.LABEL, row['name']))
    buffer.add_triple_to_buffer((being_uri, f.age, row['age']))
    buffer.add_triple_to_buffer((being_uri, f.race, row['race']))

buffer.flush_triples()

We can now examine the data in Gruff

![beings-etl](images/beings-etl.png)

Next, we import a CSV file consisting of different languages

In [6]:
#languages_df = pd.read_csv('data/beings-languages.csv')
languages_df = pd.read_csv('https://raw.githubusercontent.com/franzinc/agraph-tutorials/master/data/beings-languages.csv')

In [7]:
languages_df.head()

,id,language
0,1,Hobbitish
1,2,Sindarin
2,3,Khuzdul
3,4,Westron


We convert these to triples as well, and add to the existing buffer

In [19]:
for index, row in languages_df.iterrows():
    language_uri = conn.createURI(f"http://franz.com/example-language/{str(row['id'])}")
    buffer.add_triple_to_buffer((language_uri, RDF.TYPE, f.Language))
    buffer.add_triple_to_buffer((language_uri, RDFS.LABEL, row['language']))
    buffer.add_triple_to_buffer((language_uri, f.languageName, row['language']))
    
buffer.flush_triples()

We now examine the triples in Gruff again and see there is no connection between the languages and the beings who might speak those languages

![languages-etl](images/languages-etl.png)

We create a dataframe that will match beings to languages 

In [8]:
#speakers_df = pd.read_csv('data/beings-languages-mapping.csv')
speakers_df = pd.read_csv('https://raw.githubusercontent.com/franzinc/agraph-tutorials/master/data/beings-languages-mapping.csv')
speakers_df.head(15)

,being_id,language_id
0,1,2
1,1,4
2,2,1
3,2,4
4,3,2
5,3,4
6,4,3
7,4,4
8,5,1
9,5,2


We use the IDs to recreate the URIs that already exist in the graph to create the connections!

In [24]:
for index, row in speakers_df.iterrows():
    buffer.add_triple_to_buffer((conn.createURI(f"http://franz.com/example-being/{str(row['being_id'])}"),
                                 f.speaks,
                                 conn.createURI(f"http://franz.com/example-language/{str(row['language_id'])}")))
buffer.flush_triples()

Now looking in Gruff we see there is a connection between the beings and languages

![beings-languages-etl](images/beings-languages-etl.png)

### Add more depth with taxonomies! 
We want to know more information about all these languages. To do this we add a previously created taxonomy about Tolkien's various languages and then connect it to our existing data in the graph

In [ ]:
#conn.addFile('data/tolkien-languages.nt') #Use this if running notebook in Jupyter

#use the following code if in Google Colab
import requests
url = 'https://raw.githubusercontent.com/franzinc/agraph-tutorials/master/data/tolkien-languages.nt'
response = requests.get(url)
with open('tolkien-languages.nt', 'w') as f: f.write(response.text)
conn.addFile('tolkien-languages.nt')

![languages-taxonomy](images/languages-taxonomy.png)

Now we will connect the languages spoken by the various characters to the taxonomy. To do this we first create a mapping from language name to the ID of that concept in the taxonomy.

In [26]:
query_string = """select ?concept ?label where {
                    { ?concept skos:prefLabel ?label . } UNION { ?concept skos:altLabel ?label } }"""
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()
languages = {}
with result:
    for binding_set in result:
        concept = binding_set.getValue('concept')
        label = binding_set.getValue('label').label
        languages.update({label: concept})
pprint(languages)

{'Adûnic': <https://franz.com/example-taxonomy/lotr-languages/ed0b1aba-c6ef-47c6-b345-3df61a3cbdf4>,
 'Avarin': <https://franz.com/example-taxonomy/lotr-languages/7602d400-02a1-410d-b008-c93c00018bb9>,
 'Common Eldarin': <https://franz.com/example-taxonomy/lotr-languages/7f6f4ea8-53d4-45ec-b784-e23010d23b02>,
 'Common Telerin': <https://franz.com/example-taxonomy/lotr-languages/bc0e54b2-d6f4-4d31-b535-75c46b11d8ab>,
 'Common Tongue': <https://franz.com/example-taxonomy/lotr-languages/6f563122-6566-4b90-9e75-c6e11a3b85e1>,
 'Dalish': <https://franz.com/example-taxonomy/lotr-languages/d9e87fca-4716-493c-8761-267afe99e464>,
 'Drúedain of Brethil': <https://franz.com/example-taxonomy/lotr-languages/96063831-58a4-4a56-82e5-1e85e9353823>,
 'Drûg Languages': <https://franz.com/example-taxonomy/lotr-languages/24b23271-fa90-4c0c-aac8-f89c112e6f75>,
 'Dunlending': <https://franz.com/example-taxonomy/lotr-languages/0bc231b1-3f6b-4589-9024-14cc3ae4928b>,
 'Easterlings': <https://franz.com/example-

In [27]:
languages.get('Westron')

<https://franz.com/example-taxonomy/lotr-languages/6f563122-6566-4b90-9e75-c6e11a3b85e1>

Now we grab each language per character and connect it to the taxonomy using the `skos:exactMatch` method.

In [28]:
query_string = """select ?language ?languageName where {
                    ?language <http://franz.com/example-etl/languageName> ?languageName }"""
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()

skosExactMatch = conn.createURI('http://www.w3.org/2004/02/skos/core#exactMatch')

with result:
    for binding_set in result:
        language = binding_set.getValue('language')
        language_name = binding_set.getValue('languageName').label
        matching_language_uri = languages.get(language_name)
        buffer.add_triple_to_buffer((language, skosExactMatch, matching_language_uri))
buffer.flush_triples()

![beings-with-taxonomy-languages](images/beings-with-taxonomy-languages.png)

Now as an example let's write a query that searches for all beings that speak an `Elvish Language`. Note that all languages are also part of a `Collection` of their broadest concept. First we have the visual query.

![elvish-languages-query](images/elvish-languages-query.png)

In [29]:
query_string = """select ?name where
{ <https://franz.com/example-taxonomy/lotr-languages/5aeb87e2-6bfb-488f-a708-07ac23d73bfe> skos:member ?taxonomyLanguage .
  ?beingLanguage skos:exactMatch ?taxonomyLanguage .
  ?being <http://franz.com/example-etl/speaks> ?beingLanguage ;
         <http://franz.com/example-etl/name> ?name . }"""
result = conn.prepareTupleQuery(QueryLanguage.SPARQL, query_string).evaluate()
with result:
    for binding_set in result: print(binding_set.getValue('name').label)

Gandalf
Legolas
Aragorn
